In [1]:
from tensorflow.compat.v2.keras import backend as K
K.set_learning_phase(0)

In [2]:
from tensorflow.keras.models import load_model
model = load_model('./baseline_vgg_imagenet.h5')

W0506 13:33:36.293589 140350688552768 hdf5_format.py:224] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [3]:
print(model.outputs)
print(model.inputs)

[<tf.Tensor 'predictions/Softmax:0' shape=(None, 1000) dtype=float32>]
[<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>]


In [4]:
import tensorflow as tf

In [5]:
tf.keras.experimental.export_saved_model(model, './models/save_model_baseline_imagenet')

W0506 13:33:54.144106 140350688552768 deprecation.py:323] From /home/cody/miniconda3/lib/python3.6/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:201: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [5]:
import tensorflow as tf

def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph


frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])

AttributeError: module 'tensorflow.python.keras.api._v2.keras.backend' has no attribute 'get_session'

In [2]:
# Import TensorFlow and TensorRT
import tensorflow as tf
import tensorrt as trt
# Inference with TF-TRT `SavedModel` workflow:

ModuleNotFoundError: No module named 'tensorrt'

In [ ]:
graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:
        # Create a TensorRT inference graph from a SavedModel:
        trt_graph = trt.create_inference_graph(
            input_graph_def=None,
            outputs=None,
            input_saved_model_dir=”,
            input_saved_model_tags=[”your_saved_model_tags”],
            max_batch_size=your_batch_size,
            max_workspace_size_bytes=max_GPU_mem_size_for_TRT,
            precision_mode=”your_precision_mode”) 
        # Import the TensorRT graph into a new graph and run:
        output_node = tf.import_graph_def(
            trt_graph,
            return_elements=[“your_outputs”])
       sess.run(output_node)